# TẠO CÁC TẬP DỮ LIỆU TRAIN, TEST (SPLITS)

1. Yêu cầu chung: Tạo ra các splits, mỗi split tương ứng với một tập dữ liệu train - test
  + Bài học lý thuyết (để trả lời cho các câu hỏi như vì sao phải cần việc này, thực hiện việc này như thế nào):
    - https://www.kaggle.com/code/satishgunjal/tutorial-k-fold-cross-validation
    - https://machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/
    - https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/

  <IMG SRC = 'https://raw.githubusercontent.com/satishgunjal/images/master/KFold_Cross_Validation.png'>
2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Yamaha, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/1Uj0V9URNHpzSHeXHSB89AoGCjGki8Yra
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước
    + Số splits NumSplits - mặc định NumSplits=5 (tương đương 5-fold CV)
- Output:
    + File CarDataset.csv - Tập tin chứa tất cả ảnh của dataset
      - Chương trình sẽ scan qua cây thư mục để tìm tất cả các ảnh (chỉ chọn các ảnh có định dạng + phần mở rộng là .jpg)
      - Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
            - ImageFullPath ở dạng <Thư mục Hiệu xe>/<Tên ảnh>. Ví dụ: Honda/2024123.Honda.1.jpg.
            - CategoryID là số nguyên thuộc [0..5] theo quy ước
              - 0: Others
              - 1: Honda
              - 2: Suzuki
              - 3: Yamaha
              - 4: VinFast     
    + File CarDataset-Splits-[1..5]-[Train/Test].csv - Phân chia thành các splits, mỗi split gồm các ảnh được chia thành thành 2 tập Train - Test
      + Chương trình sẽ đọc dữ liệu từ file CarDataset.csv, sau đó với mỗi hiệu xe, chia ngẫu nhiên thành 5 tập dữ liệu. Lưu ý là phải chia theo hiệu xe, để đảm bảo dữ liệu Train/Test có dữ liệu của các hiệu xe.
      + Từ 5 tập dữ liệu chia ngẫu nhiên theo các hiệu xe Xij (i là thứ tự tập dữ liệu, j là CategoryID), gom lại thành 5 tập dữ liệu lớn hơn Xi, sao cho mỗi tập dữ liệu này chứa đủ dữ liệu của tất cả các hiệu xe. Tức là Xi = Union(Xij)       
      + Từ 5 tập dữ liệu Xi này (tương ứng với FOLDi ở trong hình vẽ trên), tạo ra 5 splits Split-i
      + Với mỗi bộ dữ liệu Split-i, ghi xuống thành 2 tập tin tương ứng với Train, Test. Ví dụ Split-1 thì ghi thành tập tin  CarDataset-Splits-1-Train.csv và CarDataset-Splits-1-Test.csv. Tập Train gồm 4 bộ, Test gồm bộ còn lại. Ví dụ,
          + Split-1: tập Train sẽ gồm X2, X3, X4, X5, tập Test là X1
          + Split-5: tập Train sẽ gồm X1, X2, X3, X4, tập Test là X5
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
- Lưu ý:
  - Nên viết thêm các cell
    - Hiển thị danh sách các tên tập tin ảnh trong từng Split-Train/Test,
    - Thống kê các ảnh cho từng CategoryID trong mỗi Split-Train/Test
  - Cần có chú thích
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: TBA
4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.

HN \\
Trần Trọng Nhân 21522924 \\
Nguyễn Văn Đức Huy  21520930 \\

## Thông tin của tác giả, ngày cập nhật

In [1]:
import os
import re
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import KFold

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
num_splits = 5

base_dir = './drive/MyDrive/Public'
dataset_dir = './drive/MyDrive/Dataset'

categories = ['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast']

def get_indexing(categories):
    indexing = {category: idx for idx, category in enumerate(categories)}
    invert_indexing = {idx: category for category, idx in indexing.items()}
    return indexing, invert_indexing

indexing, invert_indexing = get_indexing(categories)

In [4]:
def get_dataset(base_dir='./',
                dataset_dir='/',
                categories=['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast'],
                save_csv=False,
                file_name='CarDataset.csv',
                ) -> pd.DataFrame:
    os.makedirs(dataset_dir, exist_ok=True)  # Tạo thư mục đầu ra nếu chưa tồn tại

    path_list = []  # Lưu đường dẫn đầy đủ của hình ảnh
    categoryid_list = []  # Lưu mã danh mục tương ứng với từng hình ảnh


    student_ids_pattern = r'(\d{8}(?:-\d{8})*)' # Lấy MSSV hợp lệ (đủ 8 số)
    categories_pattern = '|'.join(categories) # Lấy hiệu xe hợp lệ
    file_extension_pattern = r'\.\d+\.(jpg|jpeg|png)$' # Lấy extension hợp lệ (chỉ chấp nhận file .jpg, .jpeg và .png)
    # Regex lấy tên file hợp lệ
    accepted_filename = re.compile(fr'{student_ids_pattern}\.({categories_pattern}){file_extension_pattern}')

    for category in categories: # Duyệt qua các hiệu xe
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path): # Kiểm tra nếu thư mục tồn tại
            for filename in os.listdir(category_path):
                match = accepted_filename.match(filename)
                if match: # Chỉ xử lý file có tên hợp lệ
                    _, car_category, _ = match.groups()
                    if car_category in categories:
                        full_path = os.path.join(car_category, filename)
                        path_list.append(full_path)
                        categoryid_list.append(indexing[car_category])
    # Tạo DataFrame từ danh sách đường dẫn và mã hiệu xe
    df = pd.DataFrame({
        'ImageFullPath': path_list,
        'CategoryID': categoryid_list
    })

    # Lưu CSV nếu cần
    if save_csv:
        output_file = os.path.join(dataset_dir, file_name)
        df.to_csv(output_file, index=False)
        print(f"{file_name} saved to {output_file}")

    return df

In [5]:
def save_kfold_datasets(data=None,
                        num_splits=5,
                        dataset_dir='./',
                        save_csv=False,
                        file_name='CarDataset.csv',
                        random_state=42,
                        ):

    os.makedirs(dataset_dir, exist_ok=True)  # Tạo thư mục đầu ra nếu chưa tồn tại

    if data == None:
      # Đọc dữ liệu từ file CSV ban đầu
      data_path = os.path.join(dataset_dir, file_name)
      data = pd.read_csv(data_path)

    path_list = data['ImageFullPath'].values # Danh sách đường dẫn hình ảnh
    categoryid_list = data['CategoryID'].values # Danh sách mã hiệu xe tương ứng

    train_splits = []  # Danh sách chứa các DataFrame tập train
    test_splits = []  # Danh sách chứa các DataFrame tập test

    # Khởi tạo đối tượng KFold
    kfold = KFold(n_splits=num_splits, shuffle=True, random_state=random_state)

    for i, (train_index, test_index) in enumerate(kfold.split(path_list)):
        # Đường dẫn file train/test cho từng tập con
        train_file_path = os.path.join(dataset_dir, f'CarDataset-Splits-{i + 1}-Train.csv')
        test_file_path = os.path.join(dataset_dir, f'CarDataset-Splits-{i + 1}-Test.csv')

        # Tạo DataFrame cho tập train và test
        train_data = pd.DataFrame({
            'ImageFullPath': [path_list[idx] for idx in train_index],
            'CategoryID': [categoryid_list[idx] for idx in train_index]
        })
        test_data = pd.DataFrame({
            'ImageFullPath': [path_list[idx] for idx in test_index],
            'CategoryID': [categoryid_list[idx] for idx in test_index]
        })

        if save_csv:
            # Lưu DataFrame ra file CSV
            train_data.to_csv(train_file_path, index=False)
            test_data.to_csv(test_file_path, index=False)

            print(f"Train fold {i + 1} saved to {train_file_path}")
            print(f"Test fold {i + 1} saved to {test_file_path}")

        train_splits.append(train_data)
        test_splits.append(test_data)

    return train_splits, test_splits

In [6]:
df = get_dataset(base_dir=base_dir, dataset_dir=dataset_dir, categories=categories, save_csv=True)
train_splits, test_splits = save_kfold_datasets(num_splits=num_splits, dataset_dir=dataset_dir, save_csv=True, random_state=42)

CarDataset.csv saved to ./drive/MyDrive/Dataset/CarDataset.csv
Train fold 1 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-1-Train.csv
Test fold 1 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-1-Test.csv
Train fold 2 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-2-Train.csv
Test fold 2 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-2-Test.csv
Train fold 3 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-3-Train.csv
Test fold 3 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-3-Test.csv
Train fold 4 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-4-Train.csv
Test fold 4 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-4-Test.csv
Train fold 5 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-5-Train.csv
Test fold 5 saved to ./drive/MyDrive/Dataset/CarDataset-Splits-5-Test.csv


In [7]:
df

,ImageFullPath,CategoryID
0,Others/22520394-22520395.Others.129.jpg,0
1,Others/22520394-22520395.Others.130.jpg,0
2,Others/22520394-22520395.Others.131.jpg,0
3,Others/22520394-22520395.Others.132.jpg,0
4,Others/22520394-22520395.Others.133.jpg,0
...,...,...
36755,VinFast/22521027-22520195-22521060.VinFast.103...,8
36756,VinFast/22521027-22520195-22521060.VinFast.102...,8
36757,VinFast/22521027-22520195-22521060.VinFast.102...,8
36758,VinFast/22521027-22520195-22521060.VinFast.103...,8
